# Model Evaluation

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import os

import mlflow

import json
import requests
import boto3

import streamlit as st

import warnings
warnings.filterwarnings("ignore")

## Prep

In [3]:
import yaml

def load_params():
    with open("../params.yaml") as f:
        params = yaml.safe_load(f)
    return params

params = load_params()

In [4]:
def connect_database():
    """Connect to the DynamoDB database."""
    dynamodb = boto3.resource("dynamodb",
                            aws_access_key_id=st.secrets.s3.AWS_ACCESS_KEY_ID,
                            aws_secret_access_key=st.secrets.s3.AWS_SECRET_ACCESS_KEY,
                            region_name=st.secrets.s3.AWS_DEFAULT_REGION)
    table = dynamodb.Table(st.secrets.s3.DB_NAME)
    return table

table = connect_database()

user_id = 110833230122006731136 
user_item = table.get_item(Key={"user_id": int(user_id)})["Item"]

liked_idx = list(map(int, user_item["liked_idx"].keys()))
disliked_idx = list(map(int, user_item["disliked_idx"].keys()))

data = pd.read_pickle(params["model_pipeline"]["recipe_path"])

In [5]:
data

,id,recipe_title,recipe_url,ingredients,num_steps,total_time,prep_time,cook_time,custom_time,calories,...,sodium,potassium,fiber,sugar,vitamin_a,vitamin_c,calcium,iron,serving_size,image_url
0,0,Tonkotsu Ramen,https://www.justonecookbook.com/easy-tonkotsu-...,"[pork_leg_bone, pork_hock, water, garlic, ging...",35.0,380.0,60.0,130.0,60.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.justonecookbook.com/wp-content/upl...
1,1,Pan-Fried Curry Chicken,https://www.justonecookbook.com/pan-fried-curr...,"[chicken_tender, salt, black_pepper, kewpie_ma...",15.0,40.0,10.0,10.0,0.0,290.0,...,989.0,441.0,0.5,4.0,46.0,1.0,11.0,1.0,0.0,https://www.justonecookbook.com/wp-content/upl...
2,2,Homemade Udon Noodles,https://www.justonecookbook.com/udon-noodles/,"[allpurpose_flour, water, salt, potato_starch]",36.0,120.0,60.0,0.0,150.0,361.0,...,198.0,106.0,3.0,1.0,0.0,0.0,21.0,5.0,0.0,https://www.justonecookbook.com/wp-content/upl...
3,3,Tomato Egg Vermicelli Soup,https://www.justonecookbook.com/tomato-egg-ver...,"[tomato, green_scallion, egg, chicken_broth, s...",10.0,30.0,5.0,10.0,0.0,123.0,...,299.0,215.0,2.0,3.0,737.0,11.0,79.0,2.0,0.0,https://www.justonecookbook.com/wp-content/upl...
4,4,Butter Ponzu Beef,https://www.justonecookbook.com/butter-ponzu-b...,"[beef, garlic, komatsuna, maitake_mushroom, ne...",13.0,40.0,10.0,10.0,0.0,386.0,...,751.0,684.0,2.0,1.0,194.0,17.0,90.0,3.0,0.0,https://www.justonecookbook.com/wp-content/upl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,6265,Scallion Ginger Shrimp,https://thewoksoflife.com/scallion-ginger-shri...,"[shrimp, scallion, ginger, peanut_oil, shaoxin...",4.0,15.0,10.0,5.0,0.0,191.0,...,1043.0,145.0,1.0,1.0,120.0,7.2,173.0,2.6,0.0,https://thewoksoflife.com/wp-content/uploads/2...
6266,6266,Classic Peanut Butter Cake,https://thewoksoflife.com/classic-peanut-butte...,"[allpurpose_flour, baking, baking_soda, salt, ...",4.0,60.0,30.0,30.0,0.0,517.0,...,418.0,346.0,2.0,28.0,225.0,0.0,97.0,2.2,0.0,https://thewoksoflife.com/wp-content/uploads/2...
6267,6267,Frozen White Peach Mango Margaritas,https://thewoksoflife.com/frozen-white-peach-m...,"[mango, peach, lime, simple_syrup, tequila, tr...",3.0,10.0,10.0,0.0,0.0,206.0,...,593.0,234.0,2.0,28.0,691.0,23.0,9.0,1.0,0.0,https://thewoksoflife.com/wp-content/uploads/2...
6268,6268,Cantonese Chicken Salted Fish Fried Rice,https://thewoksoflife.com/cantonese-chicken-sa...,"[oil, chicken_breast, onion, chinese_saltcured...",3.0,30.0,20.0,10.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://thewoksoflife.com/wp-content/uploads/2...


In [9]:
mlflow.set_tracking_uri("http://127.0.0.1:8080")
mlflow.set_experiment("model_evaluation")

with open("../experiment_metadata.json", "r") as f:
    metadata = json.load(f)

model = mlflow.pyfunc.load_model(metadata["local_uri"])

2025/08/24 15:54:20 INFO mlflow.tracking.fluent: Experiment with name 'model_evaluation' does not exist. Creating a new experiment.


In [10]:
text = "honey, salt, pepper, chicken, beef, fish"
query = [ingredient.strip() for ingredient in text.split(",")]
data.iloc[model.predict(query)]

,id,recipe_title,recipe_url,ingredients,num_steps,total_time,prep_time,cook_time,custom_time,calories,...,sodium,potassium,fiber,sugar,vitamin_a,vitamin_c,calcium,iron,serving_size,image_url
380,380,Honey Soy Chicken,https://www.justonecookbook.com/honey-soy-sauc...,"[chicken_drumettes, salt, black_pepper, honey,...",7.0,80.0,10.0,30.0,30.0,302.0,...,410.0,195.0,1.0,6.0,180.0,1.0,15.0,1.0,0.0,https://www.justonecookbook.com/wp-content/upl...
6135,6135,Three-Ingredient Grilled Chicken Wings,https://thewoksoflife.com/three-ingredient-gri...,"[chicken_wing, salt, black_pepper]",4.0,320.0,300.0,20.0,0.0,319.0,...,880.0,232.0,1.0,1.0,210.0,1.0,20.0,1.4,0.0,https://thewoksoflife.com/wp-content/uploads/2...
5484,5484,Down A Chicken,https://thewoksoflife.com/how-to-break-down-a-...,[chicken],9.0,10.0,10.0,0.0,0.0,409.0,...,133.0,360.0,0.0,0.0,267.0,3.0,21.0,2.0,0.0,https://thewoksoflife.com/wp-content/uploads/2...
5558,5558,Chicken Thighs,https://thewoksoflife.com/how-to-debone-chicke...,[chicken_thigh],4.0,10.0,10.0,0.0,0.0,211.0,...,74.0,198.0,0.0,0.0,75.0,0.0,8.0,1.0,0.0,https://thewoksoflife.com/wp-content/uploads/2...
4576,4576,Honey Garlic Chicken Breast,https://www.recipetineats.com/honey-garlic-chi...,"[chicken_breast, salt, pepper, flour, unsalted...",9.0,12.0,4.0,8.0,0.0,323.0,...,400.0,486.0,0.0,21.0,350.0,2.0,12.0,1.0,166.0,https://www.recipetineats.com/tachyon/2018/09/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3025,3025,Chocolate Hazelnut No-Bake Cookies,https://minimalistbaker.com/chocolate-hazelnut...,"[hazelnut_meal, medjool_nour_date, cocoa, dair...",4.0,5.0,5.0,0.0,0.0,105.0,...,0.0,0.0,2.0,12.0,0.0,0.0,0.0,0.0,1.0,https://minimalistbaker.com/wp-content/uploads...
2895,2895,Chocolate Cherry Almond Milk,https://minimalistbaker.com/chocolate-cherry-a...,"[almond_breeze_original_unsweetened, medjool_d...",3.0,15.0,15.0,0.0,0.0,162.0,...,282.0,0.0,4.6,9.0,0.0,0.0,0.0,0.0,1.0,https://minimalistbaker.com/wp-content/uploads...
1418,1418,Wheat Berries,https://www.loveandlemons.com/how-to-cook-whea...,[dry_wheat_berry],2.0,50.0,10.0,40.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://cdn.loveandlemons.com/wp-content/uploa...
1492,1492,Oat Flour,https://www.loveandlemons.com/oat-flour/,[rolled_oat],2.0,2.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://cdn.loveandlemons.com/wp-content/uploa...


In [31]:
def connect_database():
    """Connect to the DynamoDB database."""
    dynamodb = boto3.resource("dynamodb",
                            aws_access_key_id=st.secrets.s3.AWS_ACCESS_KEY_ID,
                            aws_secret_access_key=st.secrets.s3.AWS_SECRET_ACCESS_KEY,
                            region_name=st.secrets.s3.AWS_DEFAULT_REGION)
    table = dynamodb.Table(st.secrets.s3.DB_NAME)
    return table

table = connect_database()

user_id = 110833230122006731136 # alfredmastann@gmail.com test user
user_item = table.get_item(Key={"user_id": int(user_id)})["Item"]

liked_idx = list(map(int, user_item["liked_idx"].keys()))
disliked_idx = list(map(int, user_item["disliked_idx"].keys()))

loaded_model = mlflow.pyfunc.load_model(model_uri=model_info.model_uri)
predicted = loaded_model.predict([liked_idx, disliked_idx])

In [ ]:
liked_df = pd.DataFrame().from_dict(user_item["liked_idx"], orient="index").reset_index().rename(columns={"index": "recipe_id", 0: "date"})
liked_df["date"] = pd.to_datetime(liked_df["date"])
liked_df["recipe_id"] = liked_df["recipe_id"].astype(int)
liked_df.sort_values(by="date", ascending=False, inplace=True)
liked_df.reset_index(drop=True, inplace=True)
like_test = liked_df.iloc[:-5, :]
like_test

,recipe_id,date
0,1955,2025-08-11 17:47:31
1,4381,2025-08-11 17:47:25
2,4102,2025-07-22 22:25:17
3,5601,2025-07-22 22:25:12
4,995,2025-07-22 22:25:09
5,6081,2025-07-22 22:25:05
6,3220,2025-07-22 22:25:03
7,4005,2025-07-22 22:24:59
8,1076,2025-07-22 22:24:55
9,149,2025-07-22 22:24:50


In [35]:
disliked_df = pd.DataFrame().from_dict(user_item["disliked_idx"], orient="index").reset_index().rename(columns={"index": "recipe_id", 0: "date"})
disliked_df["date"] = pd.to_datetime(disliked_df["date"])
disliked_df["recipe_id"] = disliked_df["recipe_id"].astype(int)
disliked_df.sort_values(by="date", ascending=False, inplace=True)
disliked_df.reset_index(drop=True, inplace=True)
dislike_test = disliked_df.iloc[:-5, :]
dislike_test

,recipe_id,date
0,3934,2025-08-11 17:47:27
1,4006,2025-07-22 22:25:16
2,6228,2025-07-22 22:25:15
3,219,2025-07-22 22:25:14
4,6025,2025-07-22 22:25:13
5,1053,2025-07-22 22:25:08
6,5978,2025-07-22 22:25:07
7,930,2025-07-22 22:25:06
8,3193,2025-07-22 22:25:02
9,762,2025-07-22 22:25:01


In [12]:
prediction = loaded_model.predict([like_test["recipe_id"].values.tolist(), dislike_test["recipe_id"].values.tolist()])
prediction = np.asarray(json.loads(prediction))
prediction

array([4930, 3895, 3411, 5562, 5703, 3169, 3126, 3179, 5794, 5595, 3627,
       6131, 5585,  583,   90, 5301, 3400, 3149, 1496, 4241,  569, 3214,
       5892, 5499, 5685, 3830, 3211,  735, 5575,  756])

Recall@K (K = 10)

In [8]:
recall_k = np.isin(prediction, liked_df["recipe_id"].values.tolist()).sum()/len(liked_df)
recall_k

0.023809523809523808

Precision@K (K = 10)

In [9]:
precision_k = np.isin(prediction, liked_df["recipe_id"].values.tolist()).sum()/10
precision_k

0.1

### Registering

In [5]:
# Register model if validation is successful (RUN THIS TO UPDATE THE STREAMLIT MODEL)
with mlflow.start_run():
    mlflow.register_model(
        model_uri=model_info._model_uri,
        name="word2vec_model"
    )

Registered model 'word2vec_model' already exists. Creating a new version of this model...
2025/08/17 16:07:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: word2vec_model, version 12


🏃 View run charming-cub-981 at: http://127.0.0.1:5000/#/experiments/484303514770949339/runs/9a87b537f6314c6db73922d31945c24f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/484303514770949339


Created version '12' of model 'word2vec_model'.


### API Call Tests

In [14]:
params = {"query": ["rice", "beef", "meat"]}
response = requests.post(f"http://localhost:8000/recommend/", params=params)
response.text

'"[5297, 4557, 157, 161, 33, 4726, 5797, 274, 13, 618, 140, 14, 896, 251, 45, 40, 5068, 6047, 6073, 55, 206, 4709, 637, 231, 3863, 412, 5598, 3571, 5957, 307]"'